# Notebook for network charts

---

## Housekeeping

### Import dependencies

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib as mpl

from matplotlib.animation import FuncAnimation
import matplotlib.ticker as mtick
import matplotlib.animation as animation
from matplotlib.lines import Line2D
import matplotlib.patches as patches
import matplotlib.patheffects as mpe

from chickenstats.chicken_nhl import Season, Scraper
from chickenstats.chicken_nhl.info import NHL_COLORS
import chickenstats.utilities

from dotenv import load_dotenv
from pathlib import Path

import datetime as dt

import networkx as nx

### Pandas options

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Environment variables

In [ ]:
env_path = Path("../../.env")
load_dotenv(env_path)

### Chickenstats matplotlib style

In [ ]:
plt.style.use("chickenstats")

## Scrape data

### Schedule, standings, and game IDs

In [ ]:
season = Season(2024)

In [ ]:
schedule = season.schedule()

In [ ]:
standings = season.standings

In [ ]:
game_ids = schedule.loc[schedule.game_state == "OFF"].game_id.tolist()  # [:10]
live_game_ids = schedule.loc[schedule.game_state == "LIVE"].game_id.tolist()

### Play-by-play

In [ ]:
scraper = Scraper(game_ids)

In [ ]:
pbp = scraper.play_by_play.copy(deep=True)

In [ ]:
scraper.prep_stats(level="season", teammates=True)

In [ ]:
stats = scraper.stats.copy(deep=True)

In [ ]:
def create_network_graph(data: pd.DataFrame, team: str, strengths: list):
    """Docstring."""
    conds = np.logical_and.reduce(
        [
            data.team == team,
            data.strength_state.isin(strengths),
            data.position.isin(
                ["C", "L", "R", "L/R", "L/C", "R/L", "R/C", "C/L", "C/R"]
            ),
        ]
    )

    df = data.loc[conds].reset_index(drop=True)

    players = df.player.sort_values().unique().tolist()

    concat_list = [df.player.copy(deep=True)]

    for player in players:
        conds = [
            df.player == player,
            np.logical_and(
                df.player != player,
                np.logical_or(
                    df.forwards.str.contains(player), df.defense.str.contains(player)
                ),
            ),
        ]

        values = [np.nan, df.toi]

        player_series = pd.Series(np.select(conds, values, 0), name=player)

        concat_list.append(player_series)

    df = pd.concat(concat_list, axis=1).groupby("player", as_index=False).sum()

    df = df.set_index("player", drop=True)

    df = (df - df.min()) / (df.max() - df.min()) * 75

    df = df.reset_index()

    df = df.melt(
        id_vars=["player"],
        value_vars=[x for x in df.columns if x != "player"],
        var_name="target",
        value_name="weight",
    ).rename(columns={"player": "source"})

    test = nx.from_pandas_edgelist(df, edge_attr=True)

    return test

In [ ]:
# Helper function to draw any graph
def draw_graph(G, edge_options, team, edge_labels=None):
    """Draws the graph G with the specified node and edge options.

    Parameters:
        G (networkx.Graph): The graph to be drawn.
        node_options (dict): Options for drawing nodes.
        edge_options (dict): Options for drawing edges.
        edge_labels (dict, optional): Labels for the edges. Defaults to None.
    """
    fig, ax = plt.subplots(dpi=650, figsize=(8, 5))

    # Global color properties
    graph_color = "#4986e8"
    label_color = "#ffffff"
    node_options = {
        "node_color": NHL_COLORS[team]["GOAL"],
        "node_size": 1000,
        "edgecolors": NHL_COLORS[team]["SHOT"],
        "linewidths": 2,
    }

    # Define the layout of the graph
    pos = nx.spring_layout(G, iterations=10, seed=20000)

    # Draw the nodes with the specified options
    nx.draw_networkx_nodes(G, pos, **node_options)

    # Draw the node labels with specified font properties
    nx.draw_networkx_labels(
        G,
        pos,
        font_size=8,
        font_color=NHL_COLORS[team]["SHOT"],
        font_weight="bold",
        bbox={"alpha": 0.5, "color": "white"},
    )

    # Draw the edges with the specified options
    nx.draw_networkx_edges(G, pos, **edge_options)

    # Draw edge labels if they are provided
    if edge_labels:
        nx.draw_networkx_edge_labels(
            G,
            pos,
            edge_labels=edge_labels,
            connectionstyle="arc3, rad=0.3",
            font_size=12,
            font_color=NHL_COLORS[team]["MISS"],
            bbox={"alpha": 0.5, "color": "white"},
        )

    sns.despine(left=True, bottom=True)

    return fig

In [ ]:
for idx, row in standings.iterrows():
    if row.team != "NSH":
        continue

    team = row.team

    G = create_network_graph(stats, team, ["5v5"])

    weights = nx.get_edge_attributes(G, "weight")

    edge_options = {
        "edge_color": NHL_COLORS[team]["SHOT"],
        #'width': 2.5,
        "alpha": 0.7,
        "width": [weights[edge] / 10 for edge in G.edges()],
    }

    fig = draw_graph(G, edge_options, team=team)

    fig_suptitle = f"{row.team_name} forward line combinations at 5v5"
    fig.suptitle(
        fig_suptitle,
        x=0.01,
        y=1.08,
        fontsize=11,
        fontweight="bold",
        horizontalalignment="left",
    )

    todays_date = dt.datetime.now().strftime("%Y-%m-%d")
    subtitle = f"Width of connecting line indicates time-on-ice | 2024-25 season, as of {todays_date}"
    fig.text(s=subtitle, x=0.01, y=1.02, fontsize=10, horizontalalignment="left")

    # Attribution
    attribution = f"Data & xG model @chickenandstats.com | Viz @chickenandstats.com"
    fig.text(
        s=attribution,
        x=0.99,
        y=-0.05,
        fontsize=8,
        horizontalalignment="right",
        style="italic",
    )

    # Save figure
    savepath = Path(f"./charts/{team}_forwards_network.png")
    fig.savefig(savepath, transparent=False, bbox_inches="tight")

    plt.close()